In [10]:
import os
import json

import pandas as pd
BASE_DIR = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/"

In [11]:
from collections.abc import MutableMapping

def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str ='.') -> MutableMapping:
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            if type(v) == list:
                v = str(v)
            items.append((new_key, v))
    return dict(items)

In [12]:
from ray.rllib.agents.ddpg.td3 import TD3Trainer
default_tde_config = TD3Trainer.get_default_config()
default_tde_config = flatten_dict(default_tde_config)

from ray.rllib.agents.ppo.ppo import PPOTrainer
default_ppo_config = PPOTrainer.get_default_config()
default_ppo_config = flatten_dict(default_ppo_config)

from ray.rllib.agents.sac.sac import SACTrainer
default_sac_config = SACTrainer.get_default_config()
default_sac_config = flatten_dict(default_sac_config)

from ray.rllib.agents.a3c.a3c import A3CTrainer
default_a3c_config = A3CTrainer.get_default_config()
default_a3c_config = flatten_dict(default_a3c_config)

In [ ]:
default_ppo_config

In [ ]:
default_a3c_config

In [ ]:
for k in default_a3c_config:
    if k in default_ppo_config:
        if default_ppo_config[k]!=default_a3c_config[k]:
            print(k, default_ppo_config[k], default_a3c_config[k])
    else:
        print("no", k, default_a3c_config[k])

In [ ]:
table = []
for (root, dirs, files) in os.walk(BASE_DIR):
    if "params.json" not in files:
        continue
    
    experimentName = root.replace(BASE_DIR, "")
    with open(root+"/params.json", "r") as f:
        params = json.load(f)
    newDict = flatten_dict(params)
    experimentName = experimentName.split("/")
    newDict["env"] = experimentName[0]
    newDict["algo"] = experimentName[1]
    if len (experimentName) == 4:
        newDict["experiment"] = experimentName[2]
        newDict["run"] = experimentName[3]
    elif len (experimentName) == 5:
        newDict["experiment"] = experimentName[2] + "/" + experimentName[3]
        newDict["run"] = experimentName[4]
    else:
        raise ValueError(str(experimentName))
    table.append(newDict)

In [86]:
for exp in table:
    if exp["algo"]=="TD3":
        for k,v in default_tde_config.items():
            if k not in exp:
                exp[k] = v
    elif exp["algo"]=="PPO":
        for k,v in default_ppo_config.items():
            if k not in exp:
                exp[k] = v
    elif exp["algo"]=="A3C":
        for k,v in default_a3c_config.items():
            if k not in exp:
                exp[k] = v
    elif exp["algo"]=="SAC":
        for k,v in default_sac_config.items():
            if k not in exp:
                exp[k] = v
    else:
        continue
        raise ValueError("No default config for algo: {}".format(exp["algo"]))

In [87]:
df = pd.DataFrame(table)
df.shape

(9, 176)

In [88]:
df

,env,env_config.base_env_config.add_obstacles,env_config.base_env_config.constant_follower_speed,env_config.base_env_config.early_stopping.low_reward,env_config.base_env_config.early_stopping.max_distance_coef,env_config.base_env_config.follower_sensors.LaserSensor.angle_step,env_config.base_env_config.follower_sensors.LaserSensor.available_angle,env_config.base_env_config.follower_sensors.LaserSensor.points_number,env_config.base_env_config.follower_sensors.LaserSensor.return_all_points,env_config.base_env_config.follower_sensors.LaserSensor.return_only_distances,...,entropy_coeff_schedule,clip_param,vf_clip_param,grad_clip,kl_target,vf_share_layers,env_config.base_env_config.follower_sensors.LeaderCorridor_lasers.react_to_obstacles,env_config.base_env_config.follower_sensors.LeaderCorridor_lasers.react_to_safe_corridor,env_config.base_env_config.follower_sensors.LeaderCorridor_lasers.sensor_name,env_config.base_env_config.multiple_end_points
0,env4v2,True,False,-300,1.4,5.0,100.0,30.0,False,True,...,None,0.3,10.0,None,0.01,-1,NaN,NaN,NaN,NaN
1,env4v2,True,False,-300,1.4,NaN,NaN,NaN,NaN,NaN,...,None,0.3,10.0,None,0.01,-1,True,True,LeaderCorridor_lasers,NaN
2,env4v2,True,False,-300,1.4,NaN,NaN,NaN,NaN,NaN,...,None,0.3,10.0,None,0.01,-1,NaN,NaN,NaN,NaN
3,env4,True,False,-300,1.4,NaN,NaN,NaN,NaN,NaN,...,None,0.3,10.0,None,0.01,-1,NaN,NaN,NaN,NaN
4,env7,True,False,-300,1.4,5.0,100.0,30.0,False,True,...,None,0.3,10.0,None,0.01,-1,NaN,NaN,NaN,True
5,env7,True,False,-300,1.4,NaN,NaN,NaN,NaN,NaN,...,None,0.3,10.0,None,0.01,-1,NaN,NaN,NaN,True
6,env7,True,False,-300,1.4,NaN,NaN,NaN,NaN,NaN,...,None,0.3,10.0,None,0.01,-1,True,True,LeaderCorridor_lasers,True
7,obst_dynLSpd_dynFPS,True,False,-300,1.4,NaN,NaN,NaN,NaN,NaN,...,None,0.3,10.0,None,0.01,-1,True,True,LeaderCorridor_lasers,NaN
8,obst_dynLSpd_dynFPS,True,False,-300,1.4,5.0,100.0,30.0,False,True,...,None,0.3,10.0,None,0.01,-1,NaN,NaN,NaN,NaN


In [67]:
#df = df[df["algo"]=="PPO"]

In [89]:
df.fillna("-", inplace=True)

In [90]:
nunique = df.nunique()
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)
df.shape

(9, 15)

In [91]:
#df = df.loc[:, ["algo", "experiment", "run"] + sorted([c for c in df.columns if c not in ["algo", "experiment", "run"]])]
#df = df.sort_values(["algo", "experiment"])
df = df.loc[:, ["experiment", "env", "run"] + sorted([c for c in df.columns if c not in ["algo", "experiment", "env", "run"]])]
df = df.sort_values(["experiment"])

In [92]:
df.to_csv("ExperimentsTable_ppo.csv", sep=";", encoding="cp1251", index=False)

In [27]:
EXPERIMENTS = ["/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4/PPO/feats2v2_train/ppo_env4feats2v2_train5v2/PPO_continuous-grid_f1709_00000_0_2022-05-31_15-26-49/progress.csv",
"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4feats9v3_train5v2/PPO_continuous-grid_f1709_00001_1_2022-05-31_15-27-05/progress.csv",
"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/env4feats10v7_train/ppo_env4feats10v7_train5v2/PPO_continuous-grid_f1709_00002_2_2022-05-31_15-27-22/progress.csv",

"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/env4v2feats10v7_train/ppo_env4v2feats10v7_train5v2/PPO_continuous-grid_f2066_00002_2_2022-05-31_15-27-23/progress.csv",
"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/feats2v2_train/ppo_env4v2feats2v2_train5v2/PPO_continuous-grid_f2066_00000_0_2022-05-31_15-26-50/progress.csv",
 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/feats2v2_train/ppo_env4v2feats9v3_train5v2/PPO_continuous-grid_f2066_00001_1_2022-05-31_15-27-07/progress.csv",

 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env7/PPO/feats10v7_train/ppo_env7feats10v7_train5v2/PPO_continuous-grid_dd8f1_00002_2_2022-05-31_15-26-54/progress.csv",
 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env7/PPO/feats2v2_train/ppo_env7feats2v2_train5v2/PPO_continuous-grid_dd8f1_00000_0_2022-05-31_15-26-15/progress.csv",
 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env7/PPO/feats2v2_train/ppo_env7feats9v3_train5v2/PPO_continuous-grid_dd8f1_00001_1_2022-05-31_15-26-34/progress.csv"]
import pandas as pd
for exp_path in EXPERIMENTS:
    df = pd.read_csv(exp_path)
    print(df['time_total_s'].max() / 3600)

26.552425564395058
27.454996372858684
30.7174879248937
35.583249368005326
28.113892298804387
28.693616408175888
34.659060314231446
31.302585955924453
30.980811904470123


In [35]:
EXPERIMENTS = ["/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4/PPO/feats2v2_train/ppo_env4feats2v2_train5v2/PPO_continuous-grid_f1709_00000_0_2022-05-31_15-26-49/",
"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4feats9v3_train5v2/PPO_continuous-grid_f1709_00001_1_2022-05-31_15-27-05/",
"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/env4feats10v7_train/ppo_env4feats10v7_train5v2/PPO_continuous-grid_f1709_00002_2_2022-05-31_15-27-22/",

"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/env4v2feats10v7_train/ppo_env4v2feats10v7_train5v2/PPO_continuous-grid_f2066_00002_2_2022-05-31_15-27-23/",
"/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/feats2v2_train/ppo_env4v2feats2v2_train5v2/PPO_continuous-grid_f2066_00000_0_2022-05-31_15-26-50/",
 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/feats2v2_train/ppo_env4v2feats9v3_train5v2/PPO_continuous-grid_f2066_00001_1_2022-05-31_15-27-07/",

 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env7/PPO/feats10v7_train/ppo_env7feats10v7_train5v2/PPO_continuous-grid_dd8f1_00002_2_2022-05-31_15-26-54/",
 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env7/PPO/feats2v2_train/ppo_env7feats2v2_train5v2/PPO_continuous-grid_dd8f1_00000_0_2022-05-31_15-26-15/",
 "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env7/PPO/feats2v2_train/ppo_env7feats9v3_train5v2/PPO_continuous-grid_dd8f1_00001_1_2022-05-31_15-26-34/"]
import re
import pandas as pd
for exp_path in EXPERIMENTS:
    print(exp_path)
    success_rates = []
    for fName in os.listdir(exp_path):
        if "checkpoint-" in fName and ("_test.csv" in fName or "_test_env4_2.csv" in fName):
            test_df= pd.read_csv(exp_path + "/" + fName, sep=";")
            if len(test_df) <100:
                print(fName, "less then 100 seeds tested", len(test_df))
                continue
            checkpoint_number = int(re.search("\d+", fName).group())
            success_rate = test_df["mission_status"].isin(["success", "finished_by_time"]).sum()
            #axes[1].scatter(log_df.loc[checkpoint_number]["timesteps_total"], success_rate)
            #print(checkpoint_number, success_rate)
            success_rates.append(success_rate)
    print(sum(success_rates)/len(success_rates))

/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4/PPO/feats2v2_train/ppo_env4feats2v2_train5v2/PPO_continuous-grid_f1709_00000_0_2022-05-31_15-26-49/
29.5
/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4feats9v3_train5v2/PPO_continuous-grid_f1709_00001_1_2022-05-31_15-27-05/
58.97435897435897
/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/env4feats10v7_train/ppo_env4feats10v7_train5v2/PPO_continuous-grid_f1709_00002_2_2022-05-31_15-27-22/
checkpoint-650_test.csv less then 100 seeds tested 17
26.615384615384617
/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/env4v2feats10v7_train/ppo_env4v2feats10v7_train5v2/PPO_continuous-grid_f2066_00002_2_2022-05-31_15-27-23/
29.58974358974359
/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/env4v2/PPO/feats2v2_train/ppo_env4v2feats2v2_train5v2/PPO_continuous-grid_f20

In [18]:
df.columns

Index(['episode_reward_max', 'episode_reward_min', 'episode_reward_mean',
       'episode_len_mean', 'episodes_this_iter', 'num_healthy_workers',
       'timesteps_total', 'timesteps_this_iter', 'agent_timesteps_total',
       'done', 'episodes_total', 'training_iteration', 'trial_id',
       'experiment_id', 'date', 'timestamp', 'time_this_iter_s',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'timesteps_since_restore', 'iterations_since_restore',
       'hist_stats/episode_reward', 'hist_stats/episode_lengths',
       'sampler_perf/mean_raw_obs_processing_ms',
       'sampler_perf/mean_inference_ms',
       'sampler_perf/mean_action_processing_ms',
       'sampler_perf/mean_env_wait_ms', 'sampler_perf/mean_env_render_ms',
       'timers/sample_time_ms', 'timers/sample_throughput',
       'timers/load_time_ms', 'timers/load_throughput', 'timers/learn_time_ms',
       'timers/learn_throughput', 'timers/update_time_ms',
       'info/num_steps_sampled'